In [1]:
import pandas as pd
import polars as pl
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mplfinance as mpf
from plotly.subplots import make_subplots
from dash import Dash, dcc, html
import pandas_market_calendars as mcal
nse = mcal.get_calendar('NSE')

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

polars.config.Config

In [2]:
import sys
sys.path.append('..')
from tooling.filter import find_atm, option_tool
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm
from tooling.enums import Index, AssetClass, StrikeSpread, Spot

In [3]:
bnf_spot = pd.read_csv("../data/finnifty_min.csv")
bnf_spot = pd.DataFrame(bnf_spot)
bnf_spot["datetime"] = pd.to_datetime(bnf_spot["datetime"])
bnf_spot = bnf_spot.rename(columns={'open': 'o', 'high': 'h', 'low': 'l', 'close': 'c'})
bnf_spot

,datetime,o,h,l,c,volume
0,2017-01-02 09:15:00,7442.40,7442.40,7398.05,7399.30,0
1,2017-01-02 09:16:00,7399.70,7411.95,7399.70,7407.50,0
2,2017-01-02 09:17:00,7407.70,7408.95,7385.75,7385.75,0
3,2017-01-02 09:18:00,7386.05,7389.90,7382.40,7388.90,0
4,2017-01-02 09:19:00,7389.45,7389.90,7382.05,7384.05,0
...,...,...,...,...,...,...
670271,2024-03-28 15:25:00,20997.05,21006.90,20987.75,21000.75,0
670272,2024-03-28 15:26:00,20998.35,21003.70,20990.60,20994.05,0
670273,2024-03-28 15:27:00,20994.40,21002.80,20991.00,21001.90,0
670274,2024-03-28 15:28:00,21003.15,21006.10,20988.70,20990.00,0


In [50]:
async def get_expiry(f_today):
    days_to_thursday = (1 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry

def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = '0'
    mth=expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime('%y%b').upper()
        return f'{symbol}{date_string}{strike}{opt_type}'
    else:
        if expiry.day<=9:
            date_string = f'{expiry.year - 2000}{mth}{temp}{expiry.day}'
        else :
            date_string = f'{expiry.year - 2000}{mth}{expiry.day}'
        return f'{symbol}{date_string}{strike}{opt_type}'

In [51]:
async def create_list_of_options(atm_value):
    
    # list_of_options = []
    # list_of_options.append(str(atm_value)+"CE")
    # list_of_options.append(str(atm_value)+"PE")
    # list_of_options.append(str(atm_value+100)+"CE")
    # list_of_options.append(str(atm_value+100)+"PE")
    # list_of_options.append(str(atm_value+200)+"CE")
    # list_of_options.append(str(atm_value+200)+"PE")
    # list_of_options.append(str(atm_value-100)+"CE")
    # list_of_options.append(str(atm_value-100)+"PE")
    # list_of_options.append(str(atm_value-200)+"CE")
    # list_of_options.append(str(atm_value-200)+"PE")
    # list_of_options.sort()

    
    #For nifty and FNF
    list_of_options = []
    list_of_options.append(str(atm_value)+"CE")
    list_of_options.append(str(atm_value)+"PE")
    list_of_options.append(str(atm_value+50)+"CE")
    list_of_options.append(str(atm_value+50)+"PE")
    list_of_options.append(str(atm_value+100)+"CE")
    list_of_options.append(str(atm_value+100)+"PE")
    list_of_options.append(str(atm_value-50)+"CE")
    list_of_options.append(str(atm_value-50)+"PE")
    list_of_options.append(str(atm_value-100)+"CE")
    list_of_options.append(str(atm_value-100)+"PE")
    list_of_options.sort()
    
    return list_of_options

async def create_list_of_hedges(otm_call, otm_put):
    list_of_hedges = []
    list_of_hedges.append(str(otm_call)+"CE")
    list_of_hedges.append(str(otm_put)+"PE")
    list_of_hedges.sort()

    return list_of_hedges

In [59]:
async def get_portfolio(list_of_options, list_of_hedges, f_today, f_expiry):

    portfolio = []

    # option_qty = -1 * (1000000 * 3 / entry_price)
    # hedge_qty = -5 * option_qty
    
    my_pf = {}
    # print("Hedges")
    for value in list_of_hedges:
        str_value = str(value)
        if str_value[-1] == 'P' or str_value[-1] == 'C':
            value = str_value[:-1]
        strike = int(value[:-2])
        asset_class = value[-2:-1]
        # print(f'Srike = {strike} , Asset_Class = {asset_class}')
        temp_df = {}
        temp_df[value] = await fetch_option_data(
            index=Index.FINNIFTY,
            start_date=f_today,
            start_time=dt.time(hour=9, minute=16),
            end_date=f_today,
            end_time=dt.time(hour=9, minute=45),
            expiry=f_expiry,
            strike=strike,
            asset_class=asset_class,
        )
        
        # print(temp_df)
        # print(len(temp_df[value]))
        # print(type(temp_df))
        if len(temp_df[value]) == 67 or len(temp_df[value]) == 68:
            entry_price = 10000
            entry_time = np.nan
        else:
            entry_price = temp_df[value]["o"][0]
            entry_time = temp_df[value]["datetime"][0].time()

        option_qty = -1 * (1000000 * 3 / strike)
        hedge_qty = -5 * option_qty
        
        # print(entry_time)
        my_pf["Date"] = f_today
        my_pf["Position"] = "OPEN"
        my_pf["Type"] = "HEDGE"
        my_pf["Expiry"] = f_expiry
        my_pf["Index"] = "FINNIFTY"
        my_pf["Contract"] = value
        my_pf["Entry Time"] = entry_time
        my_pf["Exit Time"] = np.nan
        my_pf["Entry Price"] = entry_price
        my_pf["Exit Price"] = np.nan
        my_pf["Quantity"] = hedge_qty
        my_pf["PnL"] = np.nan
        
        portfolio.append(my_pf.copy())
    # print("Straddles")
    for value in list_of_options:
        str_value = str(value)
        if str_value[-1] == 'P' or str_value[-1] == 'C':
            value = str_value[:-1]
        
        strike = int(value[:-2])
        asset_class = value[-2:-1]
        # print(f'Srike = {strike} , Asset_Class = {asset_class}')
        temp_df = {}
        temp_df[value] = await fetch_option_data(
            index=Index.FINNIFTY,
            start_date=f_today,
            start_time=dt.time(hour=9, minute=16),
            end_date=f_today,
            end_time=dt.time(hour=9, minute=45),
            expiry=f_expiry,
            strike=strike,
            asset_class=asset_class,
        )
        
        # print(temp_df)
        # print(len(temp_df[value]))
        # print(type(temp_df[value]))
        if len(temp_df[value]) == 67 or len(temp_df[value]) == 68:
            entry_price = 10000
            entry_time = np.nan
        else:
            entry_price = temp_df[value]["o"][0]
            entry_time = temp_df[value]["datetime"][0].time()

        option_qty = -1 * (1000000 * 3 / strike)
        hedge_qty = -5 * option_qty
        
        # print(entry_time)
        my_pf["Date"] = f_today
        my_pf["Position"] = "OPEN"
        my_pf["Type"] = "STRADDLE"
        my_pf["Expiry"] = f_expiry
        my_pf["Index"] = "FINNIFTY"
        my_pf["Contract"] = value
        my_pf["Entry Time"] = entry_time
        my_pf["Exit Time"] = np.nan
        my_pf["Entry Price"] = entry_price
        my_pf["Exit Price"] = np.nan
        my_pf["Quantity"] = option_qty
        my_pf["PnL"] = np.nan
        
        portfolio.append(my_pf.copy())
    pf = pd.DataFrame(portfolio)
    # print(pf)
    print("Get Portfolio Executed !")
    return portfolio

async def rebalance(pf, f_today, f_expiry, otm_put, otm_call):

    # option_qty = -60

    # nf_df = await fetch_spot_data(
    #         instrument=Index.FINNIFTY,
    #         start_date=f_today,
    #         end_date=f_today,
    #         start_time=dt.time(hour=9, minute=17),
    #         end_time=dt.time(hour=15,minute=22),
    #     )
    # print("...1")
    nf_df = bnf_spot.loc[((bnf_spot["datetime"].dt.date == f_today) & (bnf_spot["datetime"].dt.time >= dt.time(hour=9, minute=16))) & ((bnf_spot["datetime"].dt.time <= dt.time(hour=15,minute=22)))]
    # print(nf_df)
    # print(len(nf_df))
    # nf_df = bnf_spot.loc[(bnf_spot["datetime"].dt.date == f_today) & (bnf_spot["datetime"].dt.time == dt.time(9, 17))]
    spot = nf_df["o"].iloc[0]
    # print(spot)
    # spot = nf_df[0]["o"][0]
    
    # current_atm = round(spot/100)*100
    current_atm = round(spot/50)*50
    
    # nf_df_pandas = nf_df.to_pandas()
    # print(f"Current ATM Is : {current_atm}")
    # print("...2")
    upside_sl = otm_call
    downside_sl = otm_put

    # print(f'Upside SL = {upside_sl}')
    # print(f'Downside SL = {downside_sl}')

    for i, row in nf_df.iterrows():
        # print(row["o"])

        #SL Hit for any side
        if int(row["o"]) >= upside_sl or int(row['o']) <= downside_sl:
            pf_sl = await sl_square_off(pf, f_today, f_expiry, row["datetime"])
            return pf_sl

        elif int(row["o"]) >= current_atm+75:
            current_atm += 50
            time_of_rebalance = row["datetime"]
            new_strike = current_atm+100
            closing_strike = current_atm-150
            list_of_new_entries = []
            
            list_of_new_entries.append(str(new_strike)+"CE")
            list_of_new_entries.append(str(new_strike)+"PE")
            
            list_of_new_entries.sort()

            for value in pf:
                if str(closing_strike) in value["Contract"] and value["Position"] == "OPEN":
                    exit_df = await fetch_option_data(
                        index=Index.FINNIFTY,
                        start_date=row["datetime"].date(),
                        start_time=(row["datetime"] - dt.timedelta(minutes=30)).time(),
                        end_date=row["datetime"].date(),
                        end_time=row["datetime"].time(),
                        expiry=f_expiry,
                        strike=closing_strike,
                        asset_class=value["Contract"][-2:-1]
                    )
                    # print(exit_df)
                    if len(exit_df) == 67 or len(exit_df) == 68 or value["Entry Price"] == 10000:
                        exit_value = 10000
                        pnl = 0
                    else:
                        exit_value = exit_df[-1]["o"][0]
                        pnl = (exit_value - value["Entry Price"])*value["Quantity"]
                    # print(exit_value)
                    value["Position"] = "CLOSED"
                    value["Exit Time"] = row["datetime"].time()
                    value["Exit Price"] = exit_value
                    value["PnL"] = pnl
            
            my_pf = {}

            for value in list_of_new_entries:
                strike = int(value[:-2])
                asset_class = value[-2:-1]
                # print(f'Srike = {strike} , Asset_Class = {asset_class}')
                temp_df = {}
                temp_df[value] = await fetch_option_data(
                    index=Index.FINNIFTY,
                    start_date=f_today,
                    start_time=(time_of_rebalance - dt.timedelta(minutes=30)).time(),
                    end_date=f_today,
                    end_time=time_of_rebalance.time(),
                    expiry=f_expiry,
                    strike=strike,
                    asset_class=asset_class,
                )
                # print("New Entries of RBL +++")
                # print(temp_df[value])
                if len(temp_df[value]) == 67 or len(temp_df[value]) == 68:
                    entry_price = 10000

                    option_qty = -1 * (1000000 * 3 / strike)
                    hedge_qty = -5 * option_qty
                    
                    my_pf["Date"] = f_today
                    my_pf["Position"] = "OPEN"
                    my_pf["Type"] = "REBALANCE"
                    my_pf["Expiry"] = f_expiry
                    my_pf["Index"] = "FINNIFTY"
                    my_pf["Contract"] = value
                    my_pf["Entry Time"] = np.nan
                    my_pf["Exit Time"] = np.nan
                    my_pf["Entry Price"] = 10000
                    my_pf["Exit Price"] = np.nan
                    my_pf["Quantity"] = option_qty
                    my_pf["PnL"] = 0
                else:
                    entry_price = temp_df[value]["o"][-1]
                    entry_time = temp_df[value]["datetime"][-1].time()

                    option_qty = -1 * (1000000 * 3 / strike)
                    hedge_qty = -5 * option_qty
                    
                    my_pf["Date"] = f_today
                    my_pf["Position"] = "OPEN"
                    my_pf["Type"] = "REBALANCE"
                    my_pf["Expiry"] = f_expiry
                    my_pf["Index"] = "FINNIFTY"
                    my_pf["Contract"] = value
                    my_pf["Entry Time"] = entry_time
                    my_pf["Exit Time"] = np.nan
                    my_pf["Entry Price"] = entry_price
                    my_pf["Exit Price"] = np.nan
                    my_pf["Quantity"] = option_qty
                    my_pf["PnL"] = (my_pf["Exit Price"] - my_pf["Entry Price"])*my_pf["Quantity"]
                
                pf.append(my_pf.copy())

        elif int(row["o"]) <= current_atm-75:
            current_atm -= 50
            time_of_rebalance = row["datetime"]
            new_strike = current_atm-100
            closing_strike = current_atm+150
            list_of_new_entries = []
            
            list_of_new_entries.append(str(new_strike)+"CE")
            list_of_new_entries.append(str(new_strike)+"PE")
            
            list_of_new_entries.sort()
            # print(list_of_new_entries)

            for value in pf:
                if str(closing_strike) in value["Contract"] and value["Position"] == "OPEN":
                    exit_df = await fetch_option_data(
                        # index=Index.FINNIFTY,
                        index=Index.FINNIFTY,
                        start_date=row["datetime"].date(),
                        start_time=(row["datetime"] - dt.timedelta(minutes=30)).time(),
                        end_date=row["datetime"].date(),
                        end_time=row["datetime"].time(),
                        expiry=f_expiry,
                        strike=closing_strike,
                        asset_class=value["Contract"][-2:-1]
                    )
                    # print(exit_df)
                    if len(exit_df) == 67 or len(exit_df) == 68 or value["Entry Price"] == 10000:
                        exit_value = 10000
                        pnl = 0
                    else:
                        exit_value = exit_df[-1]["o"][0]
                        pnl = (exit_value - value["Entry Price"])*value["Quantity"]
                    # print(exit_value)
                    value["Position"] = "CLOSED"
                    value["Exit Time"] = row["datetime"].time()
                    value["Exit Price"] = exit_value
                    value["PnL"] = pnl
            
            my_pf = {}

            for value in list_of_new_entries:
                strike = int(value[:-2])
                asset_class = value[-2:-1]
                # print(f'Srike = {strike} , Asset_Class = {asset_class}')
                temp_df = {}
                temp_df[value] = await fetch_option_data(
                    # index=Index.FINNIFTY,
                    index=Index.FINNIFTY,
                    start_date=f_today,
                    start_time=(time_of_rebalance - dt.timedelta(minutes=30)).time(),
                    end_date=f_today,
                    end_time=time_of_rebalance.time(),
                    expiry=f_expiry,
                    strike=strike,
                    asset_class=asset_class,
                )
                # print("New Entries of RBL +++")
                # print(temp_df[value])
                if len(temp_df[value]) == 67 or len(temp_df[value]) == 68:
                    entry_price = 10000

                    option_qty = -1 * (1000000 * 3 / strike)
                    hedge_qty = -5 * option_qty
                    
                    my_pf["Date"] = f_today
                    my_pf["Position"] = "OPEN"
                    my_pf["Type"] = "REBALANCE"
                    my_pf["Expiry"] = f_expiry
                    my_pf["Index"] = "FINNIFTY"
                    my_pf["Contract"] = value
                    my_pf["Entry Time"] = np.nan
                    my_pf["Exit Time"] = np.nan
                    my_pf["Entry Price"] = 10000
                    my_pf["Exit Price"] = np.nan
                    my_pf["Quantity"] = option_qty
                    my_pf["PnL"] = 0
                else:
                    entry_price = temp_df[value]["o"][-1]
                    entry_time = temp_df[value]["datetime"][-1].time()

                    option_qty = -1 * (1000000 * 3 / strike)
                    hedge_qty = -5 * option_qty
                    
                    my_pf["Date"] = f_today
                    my_pf["Position"] = "OPEN"
                    my_pf["Type"] = "REBALANCE"
                    my_pf["Expiry"] = f_expiry
                    my_pf["Index"] = "FINNIFTY"
                    my_pf["Contract"] = value
                    my_pf["Entry Time"] = entry_time
                    my_pf["Exit Time"] = np.nan
                    my_pf["Entry Price"] = entry_price
                    my_pf["Exit Price"] = np.nan
                    my_pf["Quantity"] = option_qty
                    my_pf["PnL"] = (my_pf["Exit Price"] - my_pf["Entry Price"])*my_pf["Quantity"]
                
                pf.append(my_pf.copy())
    # print(pf)
    print("Rebalance Executed !")
    
    square_off_portfolio = await square_off(pf, f_today, f_expiry)
    return square_off_portfolio

async def sl_square_off(pf, f_today, f_expiry, sl_hit_timestamp):
    for value in pf:
        
        if value["Position"] == "OPEN":
            closing_strike = value["Contract"][:5]
            if str(closing_strike)[-1] == 'P' or str(closing_strike)[-1] == 'C':
                closing_strike = closing_strike[:-1]
            closing_strike = int(closing_strike)
            exit_df = await fetch_option_data(
                # index=Index.FINNIFTY,
                index=Index.FINNIFTY,
                start_date=f_today,
                start_time=(sl_hit_timestamp - dt.timedelta(minutes=30)).time(),
                end_date=f_today,
                end_time=sl_hit_timestamp.time(),
                expiry=f_expiry,
                strike=closing_strike,
                asset_class=value["Contract"][-2:-1]
            )
            # print(exit_df)
            # print(f'Length of Recieved DataFrame is : {len(exit_df)}')
            if len(exit_df) == 67 or len(exit_df) == 68:
                exit_value = 10000
                exit_time = np.nan
                pnl = 0
            else:
                exit_value = exit_df[0]["o"][-1]
                exit_time = sl_hit_timestamp.time()
                pnl = (exit_value - value["Entry Price"])*value["Quantity"]
            value["Position"] = "SL CLOSED"
            value["Exit Time"] = exit_time
            value["Exit Price"] = exit_value
            value["PnL"] = pnl
    portfolio = pd.DataFrame(pf)
    # print(portfolio)
    print("SL Square Off Executed ! ")
    return portfolio

async def square_off(pf, f_today, f_expiry):
    # print("PF :")
    # print(pf)
    # print(type(pf))
    for value in pf:
        # print(value)
        if value["Position"] == "OPEN":
            closing_strike = value["Contract"][:5]
            if str(closing_strike)[-1] == 'P' or str(closing_strike)[-1] == 'C':
                closing_strike = closing_strike[:-1]
            closing_strike = int(closing_strike)
            exit_df = await fetch_option_data(
                # index=Index.FINNIFTY,
                index=Index.FINNIFTY,
                start_date=f_today,
                start_time=dt.time(hour=15, minute=0),
                end_date=f_today,
                end_time=dt.time(hour=15, minute=22),
                expiry=f_expiry,
                strike=closing_strike,
                asset_class=value["Contract"][-2:-1]
            )
            # print(exit_df)
            # print(f'Length of Recieved DataFrame is : {len(exit_df)}')
            if len(exit_df) == 67 or len(exit_df) == 68 or value["Entry Price"] == 10000:
                exit_value = 10000
                exit_time = np.nan
                pnl = 0
            else:
                exit_value = exit_df[0]["o"][-1]
                exit_time = dt.time(hour=15, minute=22)
                pnl = (exit_value - value["Entry Price"])*value["Quantity"]
            value["Position"] = "CLOSED"
            value["Exit Time"] = exit_time
            value["Exit Price"] = exit_value
            value["PnL"] = pnl
    portfolio = pd.DataFrame(pf)
    # print(portfolio)
    print("Square Off Executed ! ")
    return portfolio

In [60]:
async def func(f_today):

    # f_today = dt.date(year=2023, month=2, day=6)
    f_expiry = await get_expiry(f_today)
    
    # Get 9:17 a.m. SPOT Price to calculate ATM strikes
    # x = await fetch_spot_data(
    #         instrument=Index.FINNIFTY,
    #         start_date=f_today,
    #         end_date=f_today,
    #         start_time=dt.time(hour=9, minute=17),
    #         end_time=dt.time(hour=9,minute=17),
    #     )
    spot_row = bnf_spot.loc[(bnf_spot["datetime"].dt.date == f_today) & (bnf_spot["datetime"].dt.time == dt.time(9, 16))]
    x = spot_row["o"]
    # print(x)
    if len(x) == 67 or len(x) == 68:
        print("No value found !!!!!!!!!!")
        return pd.DataFrame()
    # spot = x[0]["o"][0]
    spot = x.values[0]
    # print(spot)
    
    # spot = int(round(spot / 100) * 100)
    spot = int(round(spot / 50) * 50)
    
    atm_value = spot
    # print(atm_value)
    
    # otm_call = int(round(atm_value* 1.015 / 100) * 100)
    # otm_put = int(round(atm_value* 0.985 / 100) * 100)

    otm_call = int(round(atm_value* 1.015 / 50) * 50)
    otm_put = int(round(atm_value* 0.985 / 50) * 50)
    
    # print(f"OTM Call = {otm_call}")
    # print(f"OTM Put = {otm_put}")

    # Create 5 Legs and 2 Hedges
    list_of_options = []
    list_of_options = await create_list_of_options(atm_value)
    
    list_of_hedges = []
    list_of_hedges = await create_list_of_hedges(otm_call, otm_put)
    
    portfolio = []
    portfolio = await get_portfolio(list_of_options, list_of_hedges, f_today, f_expiry)
    if len(portfolio) == 0:
        return {}
    # print("Going into RBL code")
    portfolio = await rebalance(portfolio, f_today, f_expiry, otm_put, otm_call)
    # portfolio = await square_off(portfolio, f_today, f_expiry)
    # print("Completed RBL code")
    return portfolio

list_of_avoided_dates_in_bnf = [
    dt.datetime(year=2017, month=10, day=19).date(),
    dt.datetime(year=2017, month=11, day=2).date(),
    dt.datetime(year=2018, month=2, day=12).date(),
    dt.datetime(year=2018, month=4, day=4).date(),
    dt.datetime(year=2018, month=5, day=25).date(),
    dt.datetime(year=2018, month=9, day=4).date(),
    dt.datetime(year=2018, month=10, day=25).date(),
    dt.datetime(year=2018, month=11, day=7).date(),
    dt.datetime(year=2019, month=2, day=4).date(),
    dt.datetime(year=2019, month=3, day=27).date(),
    dt.datetime(year=2019, month=5, day=17).date(),
    dt.datetime(year=2019, month=8, day=27).date(),
    dt.datetime(year=2019, month=10, day=17).date(),
    dt.datetime(year=2020, month=1, day=27).date(),
    dt.datetime(year=2020, month=3, day=18).date(),
    dt.datetime(year=2020, month=5, day=8).date(),
    dt.datetime(year=2020, month=5, day=25).date(),
    dt.datetime(year=2020, month=8, day=18).date(),
    dt.datetime(year=2020, month=10, day=8).date(),
    dt.datetime(year=2021, month=1, day=18).date(),
    dt.datetime(year=2021, month=3, day=10).date(),
    dt.datetime(year=2021, month=4, day=30).date(),
    dt.datetime(year=2021, month=8, day=10).date(),
    dt.datetime(year=2021, month=9, day=30).date(),
    dt.datetime(year=2022, month=1, day=10).date(),
    dt.datetime(year=2022, month=3, day=2).date(),
    dt.datetime(year=2022, month=4, day=22).date(),
    dt.datetime(year=2022, month=8, day=2).date(),
    dt.datetime(year=2022, month=9, day=22).date(),
    dt.datetime(year=2023, month=1, day=2).date(),
    dt.datetime(year=2023, month=2, day=22).date(),
    dt.datetime(year=2023, month=3, day=7).date(),
    dt.datetime(year=2023, month=6, day=29).date(),
    dt.datetime(year=2023, month=8, day=10).date(),
    dt.datetime(year=2023, month=10, day=19).date(),
    dt.datetime(year=2023, month=10, day=26).date(),
    dt.datetime(year=2023, month=11, day=2).date(),
    dt.datetime(year=2023, month=11, day=9).date(),
]

list_of_avoided_dates_in_nifty = [
    dt.datetime(year=2019, month=2, day=4).date(),
    dt.datetime(year=2019, month=3, day=27).date(),
    dt.datetime(year=2019, month=4, day=4).date(),
    dt.datetime(year=2019, month=4, day=11).date(),
    dt.datetime(year=2019, month=4, day=18).date(),
    dt.datetime(year=2019, month=5, day=2).date(),
    dt.datetime(year=2019, month=5, day=9).date(),
    dt.datetime(year=2019, month=5, day=17).date(),
    dt.datetime(year=2019, month=8, day=27).date(),
    dt.datetime(year=2019, month=10, day=17).date(),
    dt.datetime(year=2020, month=1, day=27).date(),
    dt.datetime(year=2020, month=3, day=18).date(),
    dt.datetime(year=2020, month=5, day=8).date(),
    dt.datetime(year=2020, month=5, day=25).date(),
    dt.datetime(year=2020, month=8, day=18).date(),
    dt.datetime(year=2020, month=10, day=8).date(),
    dt.datetime(year=2021, month=1, day=18).date(),
    dt.datetime(year=2021, month=3, day=10).date(),
    dt.datetime(year=2021, month=4, day=30).date(),
    dt.datetime(year=2021, month=8, day=10).date(),
    dt.datetime(year=2021, month=9, day=30).date(),
    dt.datetime(year=2022, month=1, day=10).date(),
    dt.datetime(year=2022, month=3, day=2).date(),
    dt.datetime(year=2022, month=4, day=22).date(),
    dt.datetime(year=2022, month=8, day=2).date(),
    dt.datetime(year=2022, month=9, day=22).date(),
    dt.datetime(year=2023, month=1, day=2).date(),
    dt.datetime(year=2023, month=2, day=22).date(),
    dt.datetime(year=2023, month=3, day=7).date(),
]

list_of_avoided_dates_in_fnf = [
    dt.datetime(year=2019, month=2, day=4).date(),
    # dt.datetime(year=2020, month=3, day=24).date(),
    # dt.datetime(year=2020, month=3, day=31).date(),
    # dt.datetime(year=2020, month=4, day=7).date(),
    
]

In [61]:
async def backtest():

    final_pf = []

    # list_of_skipped_dates = []
    nse = mcal.get_calendar('NSE')

    # print("Hello")

    st_dt = dt.date(year=2022, month=10, day=25)
    # st_dt = dt.date(year=2023, month=10, day=19)
    en_dt = dt.date(year=2024, month=3, day=31)
    # en_dt = dt.date(year=2017, month=12, day=31)

    while st_dt <= en_dt:
        expiry = await get_expiry(st_dt)
        if st_dt in list_of_avoided_dates_in_fnf:
            print(f"Date Avoided")
        elif st_dt == expiry:
        # if st_dt.weekday() in [5,6]:
        #     print(f"Weekend : {st_dt.weekday()}")
        # elif nse.valid_days(start_date=st_dt, end_date=st_dt).empty:
        #     print("Market Holiday")
        # else:
            try:
                print(st_dt)
                pf = await func(st_dt)
                
                for index, row in pf.iterrows():
                    if row['Entry Price'] == 10000 or row['Exit Price'] == 10000:
                        pf.at[index, 'PnL'] = 0

                # pf["Points"] = (pf["Entry Price"] - pf["Exit Price"]) if pf["Type"] != "HEDGE" else (pf["Exit Price"] - pf["Entry Price"])
                # pf["Points w C_S"] = pf["Points"] - (0.0025 * (pf["Entry Price"] + pf["Exit Price"])) if pf["Points"] > 0 else pf["Points"] + (0.0025 * (pf["Entry Price"] + pf["Exit Price"]))
                # pf["ROI"] = pf["Points w C_S"] * pf["Quantity"] / 1000000
                
                # print(pf)
                final_pf.append(pf)
            except KeyError or TypeError:
                pass
        st_dt = st_dt + dt.timedelta(days=1)
    tradebook = pd.concat(final_pf)
    # tb_polar = pl.DataFrame(tradebook)
    # print(tb_polar)
    tradebook.to_csv("FNF_5LS_1.csv", index=False)

await backtest()

2022-10-25
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-11-01
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-11-07
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-11-15
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-11-22
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-11-29
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-12-06
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-12-13
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-12-20
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2022-12-27
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2023-01-03
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2023-01-10
Get Portfolio Executed !
Rebalance Executed !
Square Off Executed ! 
2023-01-17
Get Portfolio Executed !
Reba

##### 